<a href="https://colab.research.google.com/github/alievk/avatarify/blob/feat%2Fcolab-mode/avatarify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Avatarify Colab Server

This Colab notebook is for running Avatarify rendering server.

The client, which can run on your laptop, connects to the server via `ngrok` TCP tunnel or a reverse `ssh` tunnel.

`ngrok`, while easy to use, induces a considerable network lag, which can vary from dozens of milliseconds to a second. This can lead to untable and poor performance.

A more stable solution would be a reverse `ssh` tunnel to a host with a public IP, like an AWS `t3.micro` (free) instance. This notebook provides a script for creating a tunnel, but launching an instance in a cloud is on your own.

#### To start the server
Run the cells below sequentially and pay attention to the hints and instructions in the comments.

At the end you will get a command for running the client.

#### To start the client

You also need the latest version of the client from the `feat/colab-mode` branch:
```
cd avatarify
git pull origin
git checkout feat/colab-mode
```
Then run the command from the notebook.


# Install

### Avatarify
Follow the steps below to clone Avatarify and install the dependencies.

In [0]:
!cd /content
!rm -rf *

In [0]:
!git clone https://github.com/alievk/avatarify.git -b feat/colab-mode

In [0]:
cd avatarify

In [0]:
!git clone https://github.com/alievk/first-order-model.git fomm
!pip install face-alignment==1.0.0 msgpack_numpy

In [0]:
!scripts/download_data.sh

### ngrok
Follow the steps below to setup ngrok. You will also need to sign up on the ngrok site and get your authtoken (free).


In [0]:
# Download ngrok
!scripts/get_ngrok.sh

# Run
Start here if the runtime was restarted after installation.

In [0]:
cd /content/avatarify

In [0]:
#!git pull origin

In [0]:
from subprocess import Popen, PIPE
import shlex
import json
import time


def run_with_pipe(command):
  commands = list(map(shlex.split,command.split("|")))
  ps = Popen(commands[0], stdout=PIPE, stderr=PIPE)
  for command in commands[1:]:
    ps = Popen(command, stdin=ps.stdout, stdout=PIPE, stderr=PIPE)
  return ps.stdout.readlines()


def get_tunnel_adresses():
  info = run_with_pipe("curl http://localhost:4040/api/tunnels")
  assert info

  info = json.loads(info[0])
  for tunnel in info['tunnels']:
    url = tunnel['public_url']
    port = url.split(':')[-1]
    local_port = tunnel['config']['addr'].split(':')[-1]
    print(f'{url} -> {local_port} [{tunnel["name"]}]')
    if tunnel['name'] == 'input':
      in_addr = url
    elif tunnel['name'] == 'output':
      out_addr = url
    else:
      print(f'unknown tunnel: {tunnel["name"]}')

  return in_addr, out_addr

In [0]:
# Input and output ports for communication
local_in_port = 5557
local_out_port = 5558

# Start the worker


In [0]:
# (Re)Start the worker
log = open('log.txt', 'w')
ps = Popen(shlex.split(f'./run.sh --is-worker --in-port {local_in_port} --out-port {local_out_port}'),
           stdout=log, stderr=log)
time.sleep(3)

This command should print lines if the worker is successfully started

In [0]:
!ps aux | grep 'python3 afy/cam_fomm.py' | grep -v grep

# Open ngrok tunnel

#### Get ngrok token
Go to https://dashboard.ngrok.com/auth/your-authtoken (sign up if required), copy your authtoken and put it below.

In [0]:
# Paste your authtoken here in quotes
authtoken = "1cBggVMC6W6m7uTd66sAfJ9BTJ8_3tmpP9TWTvw2RUpZYdPu5"

Set your region

Code | Region
--- | ---
us | United States
eu | Europe
ap | Asia/Pacific
au | Australia
sa | South America
jp | Japan
in | India

In [0]:
# Set your region here in quotes
region = "eu"

In [0]:
config =\
f"""
authtoken: {authtoken}
region: {region}
console_ui: False
tunnels:
  input:
    addr: {local_in_port}
    proto: tcp    
  output:
    addr: {local_out_port}
    proto: tcp
"""

with open('ngrok.conf', 'w') as f:
  f.write(config)

In [0]:
# (Re)Open tunnel
ps = Popen('./scripts/open_tunnel_ngrok.sh', stdout=PIPE, stderr=PIPE)
time.sleep(3)

In [0]:
# Get tunnel addresses
try:
  in_addr, out_addr = get_tunnel_adresses()
except Exception as e:
  [print(l.decode(), end='') for l in ps.stdout.readlines()]
  print("Something went wrong, reopen the tunnel")

Alternatively you can create a ssh reverse tunnel to an AWS `t3.micro` instance (it's free). It has lower latency than ngrok.

1. In your AWS console go to Services -> EC2 -> Instances -> Launch Instance;
1. Choose `Ubuntu Server 18.04 LTS` AMI;
1. Choose `t3.micro` instance type and press Review and launch;
1. Confirm your key pair and press Launch instances;
1. Go to the security group of this instance and edit inbound rules. Add TCP ports 5557 and 5558 and set Source to Anywhere. Press Save rules;
1. ssh into the instance (you can find the command in the Instances if you click on the Connect button) and add this line in the end of `/etc/ssh/sshd_config`:
```
GatewayPorts yes
```
then restart `sshd`
```
sudo service sshd restart
```
1. Copy your `key_pair.pem` by dragging and dropping it into avatarify folder in this notebook;
1. Use the command below to open the tunnel;
1. Start client with a command
```
./run_mac.sh --is-client --in-addr tcp://instace.compute.amazonaws.com:5557 --out-addr tcp://instance.compute.amazonaws.com:5558
```

In [0]:
# Open reverse ssh tunnel (uncomment line below)
# !./scripts/open_tunnel_ssh.sh key_pair.pem ubuntu@instance.compute.amazonaws.com

# Start the client
Run the cell below, it will print a command. Paste this command in your Mac terminal and run.

In [0]:
print(f'./run_mac.sh --is-client --in-addr {in_addr} --out-addr {out_addr}')

# Logs

In [0]:
!cat log.txt | head -100

In [0]:
!cat recv_worker.log | tail -100

In [0]:
!cat predictor_worker.log | tail -100

In [0]:
!cat send_worker.log | tail -100